<a href="https://colab.research.google.com/github/Heity94/01_DataScience_2021/blob/main/TWSM/Class_notes_PH/PH_homework_class6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

https://huggingface.co/course/chapter1/3?fw=pt

In [2]:
import numpy as np
#possibly replace with e.g. util.pytorch_cos_sim from sentence_transformers
from scipy.spatial.distance import cosine

from transformers import pipeline


In [3]:
#this will download the default gpt2 model, around 1.2GB (or just 530MB??)
#generator = pipeline("text-generation")
#generator("In this course, we will teach you how to")

Mask filling

In [4]:
#unmasker = pipeline("fill-mask")
#unmasker("This course will teach you all about <mask> models.", top_k=2)

### Tokenizer and models

DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

https://huggingface.co/docs/transformers/model_doc/distilbert

The abstract from the paper is the following:

> As Transfer Learning from large-scale pre-trained models becomes more prevalent in Natural Language Processing (NLP), operating these large models in on-the-edge and/or under constrained computational training or inference budgets remains challenging. In this work, we propose a method to pre-train a smaller general-purpose language representation model, called DistilBERT, which can then be fine-tuned with good performances on a wide range of tasks like its larger counterparts. While most prior work investigated the use of distillation for building task-specific models, we leverage knowledge distillation during the pretraining phase and show that it is possible to reduce the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and being 60% faster. To leverage the inductive biases learned by larger models during pretraining, we introduce a triple loss combining language modeling, distillation and cosine-distance losses. Our smaller, faster and lighter model is cheaper to pre-train and we demonstrate its capabilities for on-device computations in a proof-of-concept experiment and a comparative on-device study.

In [5]:
from transformers import AutoTokenizer

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


In [6]:
#What about slang akronyms ?
IDs = tokenizer(["OMG, this movie", "WTF is going"], padding=True, truncation=True, return_tensors="tf")
print(IDs)
print(tokenizer.tokenize("OMG"))
print(tokenizer.tokenize("LOL"))
print(tokenizer.tokenize("BERT"))
print(tokenizer.tokenize("Interdisciplinary"))
print(tokenizer.tokenize("comprehensibility"))

#notice that the full BERT tokenizer would split the following into sub words:
print(tokenizer.tokenize("transformers"))

{'input_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[  101, 18168,  2290,  1010,  2023,  3185,   102],
       [  101,  1059, 24475,  2003,  2183,   102,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0]], dtype=int32)>}
['om', '##g']
['lo', '##l']
['bert']
['interdisciplinary']
['com', '##pre', '##hen', '##sibility']
['transformers']


Notice the identical tokens for identical words

In [7]:
raw_inputs = [
    "How does the dimension of the embedding come about",
    "How do you come around?",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[ 101, 2129, 2515, 1996, 9812, 1997, 1996, 7861, 8270, 4667, 2272,
        2055,  102],
       [ 101, 2129, 2079, 2017, 2272, 2105, 1029,  102,    0,    0,    0,
           0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], dtype=int32)>}


Notice the different ways of calling the tokenizer:

In [8]:
s = "BIPM alumni are in high demand!"
tokens = tokenizer.tokenize(s)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

ids2 = tokenizer(s, padding=True, truncation=True, return_tensors="tf")
print(ids2)

[12170, 9737, 9441, 2024, 1999, 2152, 5157, 999]
{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101, 12170,  9737,  9441,  2024,  1999,  2152,  5157,   999,
          102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


#### Decoding tokens

Note that the decode method not only converts the indices back to tokens, but also groups together the tokens that were part of the same words to produce a readable sentence. 

In [9]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

bipm alumni are in high demand!


### Embedding

In [10]:
from transformers import TFAutoModel

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
model = TFAutoModel.from_pretrained(checkpoint)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [11]:
#outputs = model(inputs)
#print(outputs.last_hidden_state.shape)

(2, 13, 768)


So the output is high-dimensional (embedding of dimension 768 for each individual word!)

Let's find out if the embedding is really contextual, i.e. whether identical single words are different depending on the sentence:

In [12]:

How_embed_1 = outputs.last_hidden_state[0,0,:]
How_embed_2 = outputs.last_hidden_state[1,0,:]
print(np.sum(How_embed_1==How_embed_2))#0
#outputs.last_hidden_state[0:2,0,0:10]


print("similarity:")
print( 1 - cosine(How_embed_1, How_embed_2) ) 
#most_similar=data_

0
similarity:
0.912581205368042


### Tasks:

1. Embedding Similarity 
  * Write a function that takes as input two sentences, finds identical words, computes their contextual embeddings and prints the cosine similarity.
  * Ask interesting questions (case, punctuation,...)
  * Use longer words and find similarity between sub words

2. Load the IMBD reviews and sample 500 positive and negative reviews. 
  * Extract the embedding vectors
  * Fit a Naive Bayes classifier

### Task 1

In [13]:
def similarity_common_words(sentence1, sentence2): 
  raw_inputs = [
    sentence1,
    sentence2,
]
  words_in_common = list(set(sentence1.lower().split()).intersection(sentence2.lower().split()))
  inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
  outputs = model(inputs)
  index_sentence1 = [sentence1.lower().split().index(i) for i in words_in_common]
  index_sentence2 = [sentence2.lower().split().index(i) for i in words_in_common]
  for i, word in enumerate(words_in_common):
    How_embed_1 = outputs.last_hidden_state[0,index_sentence1[i],:]
    How_embed_2 = outputs.last_hidden_state[1,index_sentence2[i],:]
    print("Similarity:", word)
    print(1 - cosine(How_embed_1, How_embed_2)) 

In [14]:
similarity_common_words("work is live", "live is work")

Similarity: live
0.22911497950553894
Similarity: is
0.7664703130722046
Similarity: work
0.10128674656152725


In [15]:
similarity_common_words("I do not care", "I do care")

Similarity: i
0.9537032842636108
Similarity: do
0.8553025722503662
Similarity: care
0.5524778962135315


### Task 2
Load the IMBD reviews and sample 500 positive and negative reviews.

Extract the embedding vectors
Fit a Naive Bayes classifier

In [1]:
!pip install transformers

https://huggingface.co/course/chapter1/3?fw=pt

In [2]:
import numpy as np
#possibly replace with e.g. util.pytorch_cos_sim from sentence_transformers
from scipy.spatial.distance import cosine

from transformers import pipeline


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pickle 
IMDB_path = "/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/IMDB"

ReloadIMDB=False

if ReloadIMDB:
  from tensorflow.keras.datasets import imdb
  (train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

  word_index = imdb.get_word_index()
  reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
  
  N=len(train_data)
  decoded_reviews = ["" for x in range(N)]

  for j in range(N):
    decoded_reviews[j] = " ".join(
      [reverse_word_index.get(i - 3, "?") for i in train_data[j]])  
    
    N=len(test_data)
  decoded_reviews_test = ["" for x in range(N)]

  for j in range(N):
    decoded_reviews_test[j] = " ".join(
      [reverse_word_index.get(i - 3, "?") for i in test_data[j]])
    
  pickle.dump(decoded_reviews_test, open(IMDB_path + "/decoded_reviews_test.pkl", "wb"))
  pickle.dump(decoded_reviews, open(IMDB_path + "/decoded_reviews_train.pkl", "wb"))
  pickle.dump(train_labels, open(IMDB_path + "/train_labels.pkl", "wb"))
  pickle.dump(test_labels, open(IMDB_path + "/test_labels.pkl", "wb"))
else:
  decoded_reviews_test = pickle.load(open(IMDB_path + "/decoded_reviews_test.pkl", "rb"))
  decoded_reviews = pickle.load(open(IMDB_path + "/decoded_reviews_train.pkl", "rb"))
  train_labels = pickle.load(open(IMDB_path + "/train_labels.pkl", "rb"))
  test_labels = pickle.load(open(IMDB_path + "/test_labels.pkl", "rb"))


In [5]:
# Sample reviews (500 pos & 500 neg reviews)
import pandas as pd
reviews_df = pd.DataFrame([decoded_reviews, train_labels]).T
reviews_df.columns = ["review", "label"]

In [6]:
pos_reviews = reviews_df[reviews_df["label"]==1].sample(50)
neg_reviews = reviews_df[reviews_df["label"]==0].sample(50)

sample_reviews = pd.concat([pos_reviews, neg_reviews], axis=0)

In [7]:
sample_reviews["label"].value_counts()

1    50
0    50
Name: label, dtype: int64

In [8]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")
#text = "Replace me by any text you'd like."
encoded_input = tokenizer(sample_reviews.review.tolist(), padding=True, truncation=True, return_tensors='tf')
output = model(encoded_input)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
output.last_hidden_state.shape

TensorShape([100, 512, 768])

In [11]:
from sklearn.naive_bayes import MultinomialNB

In [15]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
output.last_hidden_state.reshape(100,512*768)

<tf.Tensor: shape=(100, 393216), dtype=float32, numpy=
array([[-0.4639352 , -0.34315842,  0.3890403 , ..., -0.2493155 ,
        -0.45264718, -0.2762106 ],
       [-0.03637165,  0.00357627,  0.36077356, ...,  0.08964637,
         0.44046456, -0.09319736],
       [-0.08678686, -0.31876284,  0.18888038, ...,  0.15907845,
         0.09382807, -0.28075856],
       ...,
       [-0.09061913,  0.02977815,  0.18268013, ..., -0.07719649,
         0.16959175, -0.404427  ],
       [-0.10033323,  0.22673324,  0.02108204, ..., -0.17467722,
        -0.4322721 , -0.38697416],
       [-0.38555938, -0.52579784,  0.61203086, ..., -0.20721212,
        -0.14508834, -0.2929266 ]], dtype=float32)>

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = output.last_hidden_state.reshape(100,512*768)
X_train_scaled = scaler.fit_transform(X_train)

In [21]:
y_train = sample_reviews["label"].tolist()

In [22]:
clf = MultinomialNB()
clf.fit(X_train_scaled, y_train)

MultinomialNB()

In [23]:
from sklearn.metrics import accuracy_score, classification_report
# Predict the Test set results, determine accuracy
#y_pred_s = clf.predict(X_train_scaled)
print('Train accuracy: ', clf.score(X_train_scaled, y_train))


Train accuracy:  0.79


In [ ]:
# Show the confusion matrix
print(classification_report(y_test, y_pred_s))